<a href="https://colab.research.google.com/github/A4zu/C-programming-/blob/main/Personal_Finance_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:


!pip install gradio scikit-learn pandas matplotlib --quiet

import gradio as gr
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

def predict_expenses(csv_file):
    try:
        df = pd.read_csv(csv_file.name)
    except:
        return "CSV faylını düzgün yükləyin (Date və Amount sütunları olmalıdır)", None

    if 'Date' not in df.columns or 'Amount' not in df.columns:
        return "CSV-də 'Date' və 'Amount' sütunları olmalıdır", None

    # Preprocessing
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    df['Day_Num'] = (df['Date'] - df['Date'].min()).dt.days

    X = df['Day_Num'].values.reshape(-1,1)
    y = df['Amount'].values

    # Train model
    model = LinearRegression()
    model.fit(X, y)

    # Predict next 30 days
    future_days = np.array([df['Day_Num'].max() + i for i in range(1,31)]).reshape(-1,1)
    future_pred = model.predict(future_days)
    future_dates = [df['Date'].max() + timedelta(days=i) for i in range(1,31)]

    # Recommendation
    avg_expense = df['Amount'].mean()
    advice = f"Average daily expense: ${avg_expense:.2f}\n"
    if avg_expense > 100:
        advice += "⚠️ Your daily expenses are high. Consider saving more!"
    else:
        advice += "✅ Your spending looks under control."

    # Plot
    fig, ax = plt.subplots(figsize=(8,4))
    ax.plot(df['Date'], df['Amount'], label='Actual')
    ax.plot(future_dates, future_pred, linestyle='--', label='Predicted')
    ax.set_xlabel("Date")
    ax.set_ylabel("Expense Amount")
    ax.set_title("Expense Prediction")
    ax.legend()
    plt.tight_layout()

    return advice, fig

# Gradio interface
interface = gr.Interface(
    fn=predict_expenses,
    inputs=gr.File(label="Upload your expense CSV"),
    outputs=[gr.Textbox(label="Advice"), gr.Plot(label="Prediction Plot")],
    title="💰 Personal Finance & Expense Predictor",
    description="Upload a CSV with 'Date' and 'Amount' columns to get expense predictions and recommendations."
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://21a0697a339ec85791.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
